[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/slayoo/seattle-2026-workshop/blob/main/part_3_units_with_pint_and_pysdm_partial.ipynb)

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils pint PySDM mendeleev numba

# hacking aerosol-cloud μ-physics modeling concepts in Python
### (mini-workshop @Uni. Washington Seattle, Jan 2026)

### Part 3: dimensional analysis using Pint (and PySDM.physics)

notebook authors (alphabetically): 
- Sylwester Arabas
- Agnieszka Żaba
- Emma Ware

This notebook depicts:
- how physical quantities (both scalar and NumPy arrays) can be assigned with units using the [Pint](https://pint.readthedocs.io/) package and   
 the emergent improvement in code quality: avoiding unmaintainable comments, handling unit conversions, offering debugging and assertion tools  
- the resultant ability of Python interpretter to check correctness of any arithmetics involving physical quantities
- how the `DimensionalAnalysis` context manager from `PySDM.physics` enables both unit-aware and unit-unaware execution of the same code    (for compatibility with Numba just-in-time compilation, with SciPy numerical solvers, etc)
- how Pint's integration with matplotlib enables auto-labelling of plot axes and how it enforces consistent units of data plotted on the same axes

### • variables and arrays with Pint units

In [ ]:
import pint, scipy, mendeleev, numpy

def constants(si):
    return {
        "M_a": (
            0.78 * 2 * mendeleev.N.atomic_weight * si.g / si.mole + 
            0.21 * 2 * mendeleev.O.atomic_weight * si.g / si.mole + 
            0.01 * 1 * mendeleev.Ar.atomic_weight * si.g / si.mole
        ),
        "R_str": scipy.constants.R * si.J / si.K / si.mole,
        "p0": 1025 * si.hPa,
    }

rho0 = lambda c, T: c["p0"] / (c["R_str"] / c["M_a"]) / T

SI = # TODO
C = # TODO
T = # TODO

# TODO

### • dimensional analysis (at runtime, see Julia's [`Unitful.jl`](https://juliaphysics.github.io/Unitful.jl) for JIT-compile-time solution!) 

In [ ]:
# TODO

### • trying to use Pint with Scipy or Numba ... 

In [ ]:
find_T_for_rho_eq_1_kg_per_m3 = lambda x: rho0(C, T=x) - 1 * SI.kg / SI.m**3 

try:
    scipy.optimize.root_scalar(
        f=lambda x: rho0(C, T=x) - 1 * SI.kg / SI.m**3 ,
        x0=300 * SI.K
    )
except Exception as e:
    print("FAILED!", str(e))

In [ ]:
import numba

def f(x):
    return x
    
try:
    numba.jit(f)(300 * SI.K)
except Exception as e:
    print("FAILED!", str(e))

### • solution: `PySDM.physics` 

In [ ]:
from PySDM import physics

In [ ]:
si = physics.si
c = constants(si)
scipy.optimize.root_scalar(
    f=lambda x: rho0(c, T=x) - 1 * si.kg / si.m**3 ,
    x0=300 * si.K
)

In [ ]:
numba.jit(f)(300 * si.K)

In [ ]:
from PySDM.physics.dimensional_analysis import DimensionalAnalysis

with DimensionalAnalysis():
    si = physics.si
    c = constants(si)
    print(rho0(c, 300 * si.K))

### • plotting with Pint

In [ ]:
from matplotlib import pyplot
from open_atmos_jupyter_utils import show_plot

# TODO